#### Let's import the nessesary libraries

In [6]:
!pip install multiset
!pip install datasets
import pandas as pd
import math

from multiset import *
from string import punctuation
from datasets import load_dataset_builder
from datasets import load_dataset

from functools import reduce



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## The dataset

The following cells will execute the code given in the hugging face tutorial to explore the dataset
and answer the different questions around it

In [7]:
ds_builder = load_dataset_builder("rotten_tomatoes")

In [8]:
print(ds_builder.info.description)
ds_builder.info.features

Movie Review Dataset.
This is a dataset of containing 5,331 positive and 5,331 negative processed
sentences from Rotten Tomatoes movie reviews. This data was first used in Bo
Pang and Lillian Lee, ``Seeing stars: Exploiting class relationships for
sentiment categorization with respect to rating scales.'', Proceedings of the
ACL, 2005.



{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [9]:
dataset = load_dataset("rotten_tomatoes")

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
pd.Series(dataset["validation"]["label"]).value_counts()

1    533
0    533
dtype: int64

## The dataset

**1. How many splits does the dataset has? (1 point)**

They are 3 splits in the dataset.

**2. How big are these splits? (1 point)**

The nuber of in the splits are the following:
- train : 8530
- validation : 1066
- test : 1066

Each row contain a sentence and a label.

**3. What is the proportion of each class on the supervised splits? (1 point)**

The proportion of each class (neg and pos reviews) is 50 % each in every split.

## Naive Bayes classifier


## Naive Bayes classifier 
The next cells will focus on the Naive Bayes classifier.

### 1. Take a look at the data and create an adapted preprocessing function with at least
First we will preprocces the dataset, lowering every character and removing unwanted ponctuation.


In [11]:
# Let's pre-treat the input by removing the puntuation and lowering it

# We also transfer it into pandas Dataframe format to ease handling 
all_data = {}

def clean_data(text_data):
    text_data = text_data.str.lower()
    text_data = text_data.str.replace("[" + punctuation + "]( |$)", "", regex=True)
    return text_data

for type in dataset.keys():
        
    text_data = pd.Series(dataset[type]["text"])

    text_data = clean_data(text_data=text_data)
    all_data[type] = pd.concat([text_data, pd.Series(dataset[type]["label"])], axis=1)
    all_data[type].columns=["text", "label"]
    
    
data = all_data["train"]
data

,text,label
0,the rock is destined to be the 21st century's ...,1
1,the gorgeously elaborate continuation of the l...,1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,emerges as something rare an issue movie that'...,1
...,...,...
8525,any enjoyment will be hinge from a personal th...,0
8526,if legendary shlockmeister ed wood had ever ma...,0
8527,hardly a nuanced portrait of a young woman's b...,0
8528,interminably bleak to say nothing of boring,0


### 2. Implement your own naive Bayes classifier from scratch. The pseudo code can be found in the slides or the book reference.
We tokenize words with a multiset data structure (bag of word method)

In [42]:
def find_vocabulary(text_data: pd.DataFrame):
    td = text_data.copy()
    td["text"] = td["text"].apply(lambda t: Multiset(t.split(" ")))
    
    return td.groupby(by="label")["text"].sum()

V_c = find_vocabulary(data)
V = V_c.sum()

C = [0, 1]

len(find_vocabulary(data)[0])

82602

We implement the Train Naive Bayes function dicribed in the lecture

In [13]:
def train(data: pd.DataFrame, classes: list):
    V_c = find_vocabulary(data) # called bigdoc in pseudo code
    V = V_c.sum()
    
    # creates empty df where we will store our word occurence conditional probabilities 
    logLikelihood = pd.DataFrame(index=pd.Index(set(V)), columns=classes) 

    logprior = {}
    n_data = len(data)
    for c in classes:
        n_c = data["label"].value_counts()[c]
        logprior[c] = math.log(n_c / n_data)
        
        cardinal_V = (len(V) + len(V.items()))
        
        for w in V:
            count = V_c[c][w]
            logLikelihood.loc[w, c] = math.log((count + 1) / cardinal_V)
            
    return logprior, logLikelihood
            


logprior, logLikelihood = train(data, C)

In [44]:
predict_args = (logprior, logLikelihood, C, V)

#Implements Test Naive Bayes function
def predict(doc, logprior, logLikelihood, C, V):
    sumed = {}
    for c in C:
        sumed[c] = logprior[c]
        for w in doc.split(" "):
            if w in V:
                sumed[c] += logLikelihood.loc[w, c]

    return max(sumed, key=sumed.get)

# Applies prediction to a whole df
def predict_dataset(data, logprior, logLikelihood, C, V):
    data["results"] = (
        data["text"].apply(lambda t: predict(t, logprior, logLikelihood, C, V))
        == data["label"]
    )

# Print the percent of right answer
def evaluate(data, name, logprior, logLikelihood, C, V):
    print(f"results on {name} dataset : ", data["results"].value_counts()[True] / len(data))

# Predicts and evaluate on all our dataset
def accuracyScratch():
  for type in dataset.keys():
    predict_dataset(all_data[type], *predict_args)   
    evaluate(all_data[type], type, *predict_args)

### 3. Implement a naive Bayes classifier using scikit-learn.
Next cells build the Naive Bayes Classifier using sklearn

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

estimator = [('Cv', CountVectorizer()), ('Bayes', MultinomialNB())]
pipe = Pipeline(estimator)
pipe.fit(data.text, data.label)

Pipeline(steps=[('Cv', CountVectorizer()), ('Bayes', MultinomialNB())])

### 4. Report the accuracy on both training and test set, for both your implementation and the scikit-learn one.

In [46]:
print('Accuracy of our own model : \n')
accuracyScratch()
print('\nAccuracy of scikit model : \n')

for type in dataset.keys():  
    print(f"{type} : {pipe.score(all_data[type]['text'],all_data[type]['label'])}")


Accuracy of our own model : 

results on train dataset :  0.9382180539273154
results on validation dataset :  0.774859287054409
results on test dataset :  0.7917448405253283

Accuracy of scikit model : 

train : 0.9327080890973036
validation : 0.7729831144465291
test : 0.797373358348968


### 5. Most likely, the scikit-learn implementation will give better results. Looking at the documentation, explain why it could be the case.

Sklearn seems better because it uses an auto Laplace smoothing parameter, to better fit the datas.

To test this hypotheis I run the same model but with no smoothing parameter

In [43]:
estimator = [('Cv', CountVectorizer()), ('Bayes', MultinomialNB(alpha=0, force_alpha=True))]
pipeNoalpha = Pipeline(estimator)
pipeNoalpha.fit(data.text, data.label)

for type in dataset.keys():  
    print(f"{type} : {pipeNoalpha.score(all_data[type]['text'],all_data[type]['label'])}")


/usr/local/lib/python3.9/dist-packages/sklearn/naive_bayes.py:907: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


train : 0.9626025791324736
validation : 0.6857410881801126
test : 0.6951219512195121


The accuracy droped on test and was higher on train, the model seems to overfit.

The smoothing parameter is definitely important to get a better accuracy, but we should have fixed its value and not remomved it completely.

### 6. Why is accuracy a sufficient measure of evaluation here?

It is a sufficient measure because the classes or equally split.

### 7. Using one of the implementation, take at least 2 wrongly classified example from the test set and try explaining why the model failed.

In [47]:
prediction = pipe.predict(all_data["train"]['text'])

mask = prediction != all_data['train']['label']
# convert the mask to a pandas series
series = pd.Series(mask)

series.value_counts()

False    7956
True      574
Name: label, dtype: int64

### 7. Using one of the implementation, take at least 2 wrongly classified example from the test set and try explaining why the model failed.

In [55]:
wrong = all_data['train'].loc[series]

def printWrong(row : pd.Series):
    print(f"We were wrong on  : {row['text']}\nReal label : {'positive' if row['label'] else 'negative'}\n")

printWrong(wrong.iloc[10])
printWrong(wrong.iloc[12])


We were wrong on  : maybe it's just because this past year has seen the release of some of the worst film comedies in decades but honestly analyze that really isn't all that bad 
Real label : positive

We were wrong on  : day is not a great bond movie but it is a good bond movie which still makes it much better than your typical bond knock-offs 
Real label : positive



The model seems to miss classify data where negative words are used but not to describe the movie itself but other element around it, like the movies of the past few years.

### 8. [Bonus] What are the top 10 most important words (features) for each class?

In [57]:
for c in [0, 1]:
    print(f"Most impactfull words in class {c} :")
    display(logLikelihood.sort_values(by=c, ascending=False)[c].head(10))
    print("\n")

Most impactfull words in class 0 :


       -3.791026
the    -3.833919
a      -4.205771
of     -4.444657
and     -4.50111
to     -4.610328
is     -4.944625
in     -5.181166
that   -5.244806
it     -5.295343
Name: 0, dtype: object



Most impactfull words in class 1 :


       -3.795595
the    -3.831418
a      -4.096785
and    -4.195292
of     -4.246548
to     -4.769938
is     -4.914661
in      -5.15925
that   -5.227503
it     -5.363488
Name: 1, dtype: object

In [58]:
# Imports the stopword list
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
stops.add('')
print(stops)


{'', 'them', 'll', 'not', 'nor', "weren't", 'through', 'shouldn', 'ma', 'hadn', 'needn', 'i', 'our', 'few', 'wouldn', "hadn't", 'as', 'under', 'this', 'yours', "that'll", 'all', 'were', 'me', 'yourselves', 'wasn', 'themselves', 'down', 'doesn', "she's", 'but', 'out', 'won', 'isn', 'those', 'from', "you'd", 'further', 'by', 'other', 'so', "shouldn't", "couldn't", 'had', 'does', 'own', 'with', 'between', 'there', 've', 'here', 'been', 'a', 'only', "mightn't", 'their', 'her', 'itself', "needn't", 'until', 'your', 'or', "shan't", 'such', 'him', "don't", "aren't", 'than', 'shan', 'after', 'don', 'didn', 'once', 'being', 'his', 'himself', 'has', 'same', 'whom', 't', "wasn't", 'we', 'which', 'haven', "haven't", 'will', "should've", 'because', 'to', 'ourselves', 'now', "won't", 'they', 'do', 's', 'is', 'during', 'hers', 'ain', 'against', 'should', 'mustn', 'where', 'an', 'above', 'both', 'more', 'then', 'very', 'of', 'be', 'in', 'most', 'mightn', 'and', 'off', 'up', "isn't", 'at', 'she', 'thes

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:

for c in [0, 1]:
    print(f"Most impactfull words in class {c} (exepting stopwords):")
    most_impact = logLikelihood.sort_values(by=c, ascending=False)[c]
    display(most_impact.loc[~most_impact.index.isin(stops)].head(10))
    print("\n")


Most impactfull words in class 0 (exepting stopwords):


movie   -5.704595
film    -5.899681
like    -6.248482
one     -6.470226
much    -6.950068
story   -6.950068
bad     -6.984554
even    -7.063622
time    -7.149484
good    -7.149484
Name: 0, dtype: object



Most impactfull words in class 1 (exepting stopwords):


film     -5.677328
movie    -6.107624
one        -6.5022
like     -6.769631
story    -6.879193
comedy   -7.051044
good     -7.095779
even     -7.191743
funny    -7.191743
way      -7.213563
Name: 1, dtype: object

### 9. [Bonus] Play with scikit-learn's version parameters. For example, see if you can consider unigram and bigram instead of only unigrams.

In [38]:
estimator = [('Cv', CountVectorizer(ngram_range=(1,2))), ('Bayes', MultinomialNB())]
pipeBonus = Pipeline(estimator)
pipeBonus.fit(data.text, data.label)

Pipeline(steps=[('Cv', CountVectorizer(ngram_range=(1, 2))),
                ('Bayes', MultinomialNB())])

In [39]:
for type in dataset.keys():  
    print(f"{type} : {pipeBonus.score(all_data[type]['text'],all_data[type]['label'])}")


train : 0.994021101992966
validation : 0.7673545966228893
test : 0.801125703564728


### We tried multiple values for the ngram-range parameter, like only bigrams, unigram to trigrams and this one seems to be the best. But the accuracy does not raise really much

### We also tried addind the stop words, but it doesn't increase the accuracy

#**Stemming and Lemmatization**